# **Desafío #5 – API REST sobre el modelo organizacional**

## **5.1. Introducción**

Este documento desarrolla el Desafío #5, cuyo objetivo es exponer el modelo de datos organizacional mediante una API REST. La API se implementa con **FastAPI** y se construye sobre la misma base de datos sintética utilizada en los desafíos anteriores, integrando los datos de departamentos, puestos de trabajo y empleados.

La API está diseñada para:

- proporcionar endpoints que permitan consultar empleados, departamentos y métricas agregadas,  

- mantener una clara separación entre capa de datos (Data Lake) y capa de servicio,  

- y estandarizar la representación numérica, en particular los valores de tipo monetario, utilizando dos decimales, coherentes con la escala de centavos.

La implementación asume que los datos continúan almacenados en el Data Lake de Azure definido en el Desafío #3 y que existe un archivo `.env` con la configuración necesaria para acceder al Storage Account.


## **5.1 Configuración básica del entorno**

In [1]:
import sys
from pathlib import Path
from dotenv import load_dotenv

# Directorio del notebook: 01_data_cloud/docs
notebook_dir = Path.cwd()
project_root = notebook_dir.parent

# src: 01_data_cloud/src
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Cargar archivo .env ubicado en 01_data_cloud
env_path = project_root / ".env"
load_dotenv(env_path)

print("Notebook directory :", notebook_dir)
print("Project root       :", project_root)
print("Src path           :", src_path)
print(".env path          :", env_path)

Notebook directory : /Users/capeto/Library/CloudStorage/OneDrive-Personal/GitHub/mvm-technical-challenge/01_data_cloud/docs
Project root       : /Users/capeto/Library/CloudStorage/OneDrive-Personal/GitHub/mvm-technical-challenge/01_data_cloud
Src path           : /Users/capeto/Library/CloudStorage/OneDrive-Personal/GitHub/mvm-technical-challenge/01_data_cloud/src
.env path          : /Users/capeto/Library/CloudStorage/OneDrive-Personal/GitHub/mvm-technical-challenge/01_data_cloud/.env


## **5.2 Diseño general de la API**

La API se implementa como un servicio FastAPI contenido en el módulo `src/api/org_api.py`.  
El servicio se apoya en los siguientes componentes:

1. **Carga del modelo de datos desde el Data Lake**  
   Durante el arranque de la aplicación se leen los archivos Parquet almacenados en el Storage Account de Azure (contenedor `org-raw`, prefijo `org_data/v1`). A partir de estos archivos se construye una vista integrada, similar a la utilizada en el Desafío #4, que consolida la información de empleados, puestos y departamentos.

2. **Estandarización numérica**  
   Los campos de tipo monetario (por ejemplo, `salary`) y otros `float` relevantes (por ejemplo, `tenure_years`) se redondean a dos decimales antes de ser devueltos en las respuestas. Este criterio alinea la API con escenarios reales en los que se trabaja con cantidades expresadas en moneda y se evita la exposición de valores con una precisión innecesaria.

3. **Endpoints principales**  
   La API expone un conjunto acotado de endpoints orientados a lectura:

   - `GET /health`  
     Verificación básica del estado del servicio.

   - `GET /employees`  
     Listado paginado de empleados, con filtros opcionales por departamento y nivel de cargo.

   - `GET /employees/{employee_id}`  
     Detalle de un empleado específico.

   - `GET /departments`  
     Listado de departamentos y su tamaño (headcount).

   - `GET /analytics/salary-summary`  
     Resumen analítico de salarios por nivel de puesto, incluyendo promedios, mínimos y máximos redondeados a dos decimales.

Este conjunto de endpoints demuestra un patrón típico de una API analítica de datos organizacionales y sirve como base extensible para futuros requerimientos.




## **5.3 — Ejecución de la API**

El servicio FastAPI se ejecuta a partir del módulo `org_api.py` ubicado en `src/api`.

En un entorno local, dentro del directorio `01_data_cloud`, la API puede iniciarse con el siguiente comando:

```bash
uvicorn api.org_api:app --reload
````

* `api.org_api` hace referencia al módulo `src/api/org_api.py`.
* `app` es la instancia de `FastAPI` definida en dicho módulo.
* La opción `--reload` permite recargar automáticamente el servicio durante el desarrollo.

Una vez levantado el servidor, la documentación interactiva de la API queda disponible en:

* `http://localhost:8000/docs` (Swagger UI)
* `http://localhost:8000/redoc` (ReDoc)

## **5.4 Pruebas básicas desde el notebook**

Aunque la API está pensada para ser consumida por clientes externos (aplicaciones web, herramientas de análisis, etc.), es posible realizar pruebas simples desde este notebook.

En las celdas siguientes se asume que el servicio está en ejecución en `http://localhost:8000`.

In [3]:
#Prueba del healthcheck

import requests

base_url = "http://localhost:8000"

health_response = requests.get(f"{base_url}/health")
health_response.status_code, health_response.json()

(200, {'status': 'ok'})

In [4]:
#Prueba del listado de empleados

employees_response = requests.get(
    f"{base_url}/employees",
    params={"limit": 5, "offset": 0}
)
employees_response.status_code, employees_response.json()

(200,
 [{'employee_id': 1,
   'department_id': 4,
   'department_name': 'Finance',
   'job_position_id': 17,
   'job_title': None,
   'job_level': None,
   'salary': 5020.65,
   'tenure_years': 3.0,
   'age': 29},
  {'employee_id': 2,
   'department_id': 2,
   'department_name': 'Commercial',
   'job_position_id': 8,
   'job_title': None,
   'job_level': None,
   'salary': 6729.94,
   'tenure_years': 1.0,
   'age': 30},
  {'employee_id': 3,
   'department_id': 5,
   'department_name': 'Human Resources',
   'job_position_id': 24,
   'job_title': None,
   'job_level': None,
   'salary': 6857.87,
   'tenure_years': 1.0,
   'age': 57},
  {'employee_id': 4,
   'department_id': 4,
   'department_name': 'Finance',
   'job_position_id': 18,
   'job_title': None,
   'job_level': None,
   'salary': 9485.64,
   'tenure_years': 1.0,
   'age': 51},
  {'employee_id': 5,
   'department_id': 1,
   'department_name': 'Operations',
   'job_position_id': 5,
   'job_title': None,
   'job_level': None,
   

In [5]:
#Prueba del resumen salarial

salary_summary_response = requests.get(f"{base_url}/analytics/salary-summary")
salary_summary_response.status_code, salary_summary_response.json()

(200,
 {'overall_avg_salary': 6433.88,
  'overall_min_salary': 1099.27,
  'overall_max_salary': 39746.24,
  'levels': [{'job_level': 'N/A',
    'avg_salary': 6433.88,
    'min_salary': 1099.27,
    'max_salary': 39746.24,
    'count': 600}]})

## **5.5 — Cierre del Desafío #5**

La implementación presentada cumple con el objetivo del Desafío #5 al exponer el modelo organizacional a través de una API REST basada en FastAPI, utilizando como origen los datos almacenados en el Data Lake de Azure.

La solución:

- construye una vista integrada en memoria a partir de los archivos Parquet de empleados, puestos y departamentos,  

- expone endpoints claros para consulta de empleados, departamentos y métricas salariales,  

- y cuida la presentación de los valores numéricos, en particular las cantidades monetarias, utilizando redondeo a dos decimales.

Este servicio REST constituye una capa de acceso estándar que puede ser consumida por aplicaciones internas, dashboards o procesos externos, y se integra de forma natural con los componentes desarrollados en los desafíos previos.

In [2]:
import fastapi
import uvicorn
import pandas
import azure.storage.blob
import dotenv

print("Dependencias cargan correctamente.")


Dependencias cargan correctamente.
